# Part 1
## Load Data

In [7]:
import pandas as pd

train_df = pd.read_csv('train.csv', delimiter='\t')
test_df = pd.read_csv('test.csv', delimiter='\t')

# Display the first few rows
print(train_df.head())

   imageid       label                                     productname
0     2653        Bags            Murcia Women Leather Office Grey Bag
1    55997      Others  Colorbar Velvet Matte Temptation Lipstick 24MA
2     2640       Shoes           Carlton London Men Brown Formal Shoes
3    40565     Topwear                            W Women Maroon Kurta
4    38932  Bottomwear               Gini and Jony Girls Pink Leggings


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit the encoder to the labels in both train and test datasets to ensure consistency
all_labels = pd.concat([train_df['label'], test_df['label']])
label_encoder.fit(all_labels)

# Transform labels in both datasets to integer encodings
train_df['label'] = label_encoder.transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

## Data Preparation

In [9]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class FashionDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, str(self.dataframe.iloc[idx, 0]) + '.jpg')
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

### Define transformations and create data loaders

In [10]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.Grayscale(num_output_channels=3),  # Convert images to RGB
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])


# Create dataset instances
train_dataset = FashionDataset(dataframe=train_df, image_dir='archive/images', transform=transform)
test_dataset = FashionDataset(dataframe=test_df, image_dir='archive/images', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

## Simple CNN Model

My baseline CNN model is designed with simplicity and efficiency in mind, aiming to achieve reasonable accuracy while keeping computational costs low. This model serves as a starting point for further experimentation and optimization.

I start with two convolutional layers, as these are fundamental in capturing the spatial hierarchies in images. The first layer has 32 filters, and the second has 64, progressively increasing the model's capacity to learn more complex features. Each uses a kernel size of 3x3, which is common in practice for balancing the trade-off between learning fine-grained and broader features.

Following each convolutional layer, we apply max pooling with a 2x2 window to reduce the spatial dimensions of the feature maps. This downsampling technique helps to make the model more invariant to small translations of the input image.

I use the ReLU activation function for its computational efficiency and its ability to mitigate the vanishing gradient problem, helping the model learn faster and more effectively.

The model transitions from convolutional layers to fully connected layers, which perform the classification based on the features extracted by the preceding layers. The first dense layer reduces to 128 units, concentrating the information into a more compact representation, followed by the final output layer with 13 units corresponding to the number of classes, using a softmax activation to produce probability distributions over the classes.

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Assuming images are resized to 128x128
        self.fc2 = nn.Linear(128, 13)  # 13 classes

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training

In [12]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = test_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)  
                labels = labels.to(device).long()

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Initialize the model, loss function, optimizer, and learning rate scheduler
model = FashionCNN().to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
model_ft = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


  0%|          | 0/632 [00:00<?, ?it/s]

  0%|          | 0/632 [00:01<?, ?it/s]


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

## Testing

In [ ]:
def test_model(model, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()  # Set the model to evaluation mode
    corrects = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data).item()
            total += labels.size(0)

    accuracy = corrects / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    return accuracy


accuracy = test_model(model_ft, test_loader)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:05<00:00, 12.49it/s]

Test Accuracy: 94.97%


## Performance Conclusion

The model shows consistent improvement in training accuracy over the 25 epochs, starting from 86.98% in the first epoch and reaches 99.98% by the final epoch. Similarly, the training loss decreases significantly from 0.4392 in the first epoch to 0.0006 in the last epoch. This demonstrates the model's ability to learn effectively from the training data and its increasing proficiency in making accurate predictions over time. : The validation accuracy starts at a high note of 91.80% in the first epoch and sees slight improvements, peaking at 95.03% by the tenth epoch. After this point, the validation accuracy fluctuates slightly but remains relatively stable, ending at 94.97% in the final epoch. The validation loss decreases initially, reflecting the model's learning, but begins to plateau and slightly increase, suggesting the onset of overfitting to the training data. Despite this, the model maintains a high level of performance on the validation set, indicating good generalization capabilities up to a certain point.ce: The test accuracy of 94acy, sugsting that the model generalizes well to unseen data. This high level of accuracy on the test set indicates that the features learned by the model during training are robust and relevant to making accurate predictions outside the training dataset.

# Part 2

In [ ]:
from torchvision import datasets, transforms

# Define transformations for the training data and validation data
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset instances
train_dataset = FashionDataset(dataframe=train_df, image_dir='images', transform=transform)
test_dataset = FashionDataset(dataframe=test_df, image_dir='images', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model = FashionCNN().to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
model_ft = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 632/632 [00:50<00:00, 12.47it/s]


Train Loss: 0.4829 Acc: 0.8687


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:04<00:00, 13.12it/s]


Val Loss: 0.2618 Acc: 0.9158
Epoch 2/25
----------


...

Val Loss: 0.3626 Acc: 0.9505
Epoch 25/25
----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 632/632 [00:55<00:00, 11.34it/s]


Train Loss: 0.0007 Acc: 0.9998


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:04<00:00, 14.07it/s]

Val Loss: 0.3631 Acc: 0.9505
